# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path="output_data/clean_city_data.csv"
clean_city_data=pd.read_csv(path)
clean_city_data.head()


,City,Lat,Lng,Max temp (F),Humidity,Cloudiness,Wind speed,Country,Date
0,Dhidhdhoo,6.88,73.10,81.680,77,100,3.03,MV,1607286034
1,Morón,-34.65,-58.62,77.000,23,0,5.10,AR,1607286034
2,San Patricio,28.02,-97.52,64.400,48,40,7.20,US,1607286034
3,Quatre Cocos,-20.21,57.76,78.008,83,75,3.60,MU,1607285786
4,Clyde River,70.47,-68.59,3.200,91,90,5.10,CA,1607285786


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude
coordinates = clean_city_data[["Lat","Lng"]]

humidity=clean_city_data["Humidity"]


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates,weights=humidity,dissipating=False,max_intensity=100,point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# put conditions to be ideal weather 
hotel_df=clean_city_data.loc[(clean_city_data["Max temp (F)"] < 80) & (clean_city_data["Max temp (F)"]>70) & (clean_city_data["Humidity"] < 50) & (clean_city_data["Cloudiness"] == 0)].reset_index().drop(["index"],axis=1)
hotel_df

,City,Lat,Lng,Max temp (F),Humidity,Cloudiness,Wind speed,Country,Date
0,Morón,-34.65,-58.62,77.000,23,0,5.10,AR,1607286034
1,Young,-32.68,-57.63,75.992,35,0,1.34,UY,1607286100
2,Concepción del Uruguay,-32.48,-58.24,75.992,35,0,1.34,AR,1607286032
3,Tandil,-37.32,-59.13,78.998,23,0,2.24,AR,1607286492
4,Gogrial,8.53,28.10,79.358,24,0,1.40,SS,1607286556
5,Santa Rosalía,27.32,-112.28,70.286,31,0,6.29,MX,1607286558
6,Arlit,18.74,7.39,77.396,19,0,1.95,NE,1607286757


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# add new cloumn to the dataframe 
hotel_df["Hotel Name"]=""
hotel_df

,City,Lat,Lng,Max temp (F),Humidity,Cloudiness,Wind speed,Country,Date,Hotel Name
0,Morón,-34.65,-58.62,77.000,23,0,5.10,AR,1607286034,
1,Young,-32.68,-57.63,75.992,35,0,1.34,UY,1607286100,
2,Concepción del Uruguay,-32.48,-58.24,75.992,35,0,1.34,AR,1607286032,
3,Tandil,-37.32,-59.13,78.998,23,0,2.24,AR,1607286492,
4,Gogrial,8.53,28.10,79.358,24,0,1.40,SS,1607286556,
5,Santa Rosalía,27.32,-112.28,70.286,31,0,6.29,MX,1607286558,
6,Arlit,18.74,7.39,77.396,19,0,1.95,NE,1607286757,


In [81]:
base_url= "https://maps.googleapis.com/maps/api/place/textsearch/json"
radius =5000
query= 'hotel'
params = {"radius":radius,"query":query,"key":g_key}
for index,row in hotel_df.iterrows():
    lat =str(row["Lat"])
    lng=str(row["Lng"])
    location=",".join([lat,lng])
    params["location"]=location
    response=requests.get(base_url,params=params).json()
    result=response["results"]
    hotel_df.loc[index, 'Hotel Name'] = result[0]['name']
    
hotel_df

,City,Lat,Lng,Max temp (F),Humidity,Cloudiness,Wind speed,Country,Date,Hotel Name
0,Morón,-34.65,-58.62,77.000,23,0,5.10,AR,1607286034,Devoto Hotel
1,Young,-32.68,-57.63,75.992,35,0,1.34,UY,1607286100,Hotel Young
2,Concepción del Uruguay,-32.48,-58.24,75.992,35,0,1.34,AR,1607286032,Hotel Park Center
3,Tandil,-37.32,-59.13,78.998,23,0,2.24,AR,1607286492,T Design Hotel Boutique
4,Gogrial,8.53,28.10,79.358,24,0,1.40,SS,1607286556,Holiday Inn Pewaukee-Milwaukee West
5,Santa Rosalía,27.32,-112.28,70.286,31,0,6.29,MX,1607286558,Hotel Las Casitas Santa Rosalia
6,Arlit,18.74,7.39,77.396,19,0,1.95,NE,1607286757,Hôtel Telwa Bungalow


In [97]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_name=hotel_df["Hotel Name"].tolist()
marker_locations=hotel_df[["Lat","Lng"]]
fig = gmaps.figure()


# Add marker layer ontop of heat map

markers = gmaps.marker_layer(marker_locations,
[info_box_template.format(**row) for index, row in hotel_df.iterrows()])
heat_layer = gmaps.heatmap_layer(coordinates,weights=humidity,dissipating=False,max_intensity=100,point_radius=2)

# Display figure
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))